# Linear Regression

Previously, we built and evaluated a few simple machine learning models that learned from the housing data to predict sale price. These were informal models quickly put together with a few lines of code. There is nothing that prevents us from using these simple models in practice and you can justifiably make a case for each one of them. That said, there are many other machine learning models that have been devised that will produce better results.

## The linear regression model

Linear regression is a formal approach for modeling the relationship between one or more features and a continuous target variable. In **simple linear regression**, only a single feature is used to model a target variable. It is expressed as the following mathematical model.

$$\hat{y} = w_{0} + w_{1}x_{1}$$

This is simply the equation for a line where $w_{0}$ and $w_{1}$ are the y-intercept and slope, $x_{1}$ is the single feature and $\hat{y}$ is the predicted target variable (pronounced y-hat). In our housing example, the square footage would be the single feature and the sale price the target. The letter $w$ is used here to represent the word **weights**. More formally, $w_{0}$ and $w_{1}$ are called the **parameters** of the model. 

### Model sale price from above ground living area

We will use a simple linear regression to learn the relationship between the above ground living area and final sale price of the house. From the above equation, $x_1$ refers to the above ground living area, `GrLivArea`, and $\hat{y}$ refers to the predicted sale price, `SalePrice`.

### Learn the coefficients from the data

The end goal of the linear regression model is to learn the values of the parameters ($w_{0}$ and $w_{1}$) from the data. These values that it learns are often called **coefficients**.

### Difference between a parameter and a coefficient

The terms parameter and coefficient are very similar and are often used interchangeably. The parameters of a model are the unknown constants such as $w_{0}$ and $w_{1}$. The coefficients are the specific numeric values that are learned from the data. Once the coefficients are known, we use them in our model. For instance, if the coefficients are 5 and 3, our model becomes the following:

$$\hat{y} = 5 + 3x_{1}$$

### Must have measure of performance to learn from data

As we did with our simple model, we must have a way to determine what it means to improve performance. When there is an objective measure of performance, the model is able to choose the best coefficients. It is typical to use a measure of performance that relies on squared errors such as the RMSE, which we will use here.

### Algorithm on how to minimize the RMSE

There are formal algorithms that are proven to find the coefficients that minimize the RMSE. Before using them, we will come up with our own procedure for doing so. In our specific case, we need to find the values of our two parameters, $w_{0}$ and $w_{1}$.

We start by making a list of different combinations of $w_{0}$ and $w_{1}$. We can then calculate the RMSE for each combination and select the combination of coefficients that minimize this value.

### How to choose combinations $w_{0}$ and $w_{1}$?

For this simple algorithm to work, we need to create a range of possible values for each coefficient. Since we only have one feature, we can plot the relationship between above ground living area and sale price as a scatter plot and attempt to estimate the coefficient values through visual inspection. Let's read in the data and then visualize this relationship now.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
housing = pd.read_csv('../data/housing_sample.csv')
housing.head(3)

We make a scatter plot directly from our pandas DataFrame using the `plot` method.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
housing.plot(x='GrLivArea', y='SalePrice', kind='scatter', ax=ax);

## Propose search area for $w_{0}$ and $w_{1}$

We need to approximate a minimum and maximum value for both our model parameters. Using the graph above, we can manually create two lines that serve as a boundary for our parameter search. One line has a steep slope and a low y-intercept, while the other has a flat slope and a high y-intercept. Each of the following commands to the `plot` method creates a line. The x-values and y-values of two points that define each line are passed as lists. For instance, the first line connects the points (0, -200,000) to (4,000, 1,000,000).

In [ ]:
ax.plot([0, 4000], [-200_000, 1_000_000], label='steep slope, low y-intercept', color='green', lw=3)
ax.plot([0, 4000], [100_000, 200_000], label='flat slope, high y-intercept', color='red', lw=3)
ax.legend()
fig

### Calculate the y-intercept and slope for each line

The y-intercepts are -200,0000 and 100,000. The slopes for each line are $\frac{1,000,000 - {-200,000}}{4,000 - 0} = 300$ and $\frac{200,000 - 100,000}{4,000 - 0} = 25$. These values are the boundaries for the search area.

### Iterate over different combinations

Now that we have proposed bounds for $w_{0}$ and $w_{1}$, we can use the `range` function to provide a sequence of possible values. We then iterate over all these combinations calculating the RMSE. Finally, the results are stored in a pandas DataFrame where the first 10 combinations are output.

In [ ]:
X = housing['GrLivArea']
y = housing['SalePrice']
errors = {'w0': [], 'w1': [], 'rmse': []}
w0_range = range(-100_000, 125_000, 25_000)
w1_range = range(50, 300, 25)

for w0 in w0_range:
    for w1 in w1_range:
        y_pred = w0 + w1 * X
        error = y - y_pred
        rmse = np.sqrt((error ** 2).mean()).round(-3)
        errors['w0'].append(w0)
        errors['w1'].append(w1)
        errors['rmse'].append(rmse)
        
df_errors = pd.DataFrame(errors)
df_errors.head(10)

### Sort by least error

The resulting DataFrame is sorted by error from least to greatest error to find the best combination for $w_{0}$ and $w_{1}$. The five best combinations are shown below.

In [ ]:
df_errors.sort_values('rmse').head()

### The final model

The best combination of coefficients are 25,000 and 100. We now have a model with coefficients that were learned from the data. We can use this in the future to make predictions.

$$\hat{y} = 25,000 + 100x$$

## Plot the model along with the actual data

Let's define a function that makes predictions with our model and then use it to plot the predicted sale prices on top of our actual data.

In [ ]:
def simple_lr_model(X):
    return 25000 + 100 * X

X = np.array([0, 4000])
y_pred = simple_lr_model(X)
ax.plot(X, y_pred, color='black', label='best model', lw=5)
ax.set_title('Best Linear Regression Model: $\hat{y} = 25,000 + 100x1$')
ax.legend()
fig

## R-squared - a slightly different error metric

Although the RMSE is a perfectly valid measure of performance, it is difficult to interpret across different datasets. Because it involves taking the mean, it is a relative measure of performance for all the models built on the same dataset, but it is not a relative measure of performance for all datasets. Instead, it is common to go one step further and produce a measure that shows relative performance regardless of the dataset that is used. This error metric that we will now discuss is called **R-Squared**.

### Visualizing R-Squared

I believe R-Squared is best understood with a visual example. Take a look at the following images.

![][1]

[1]: images/squared_error.png

The above visualization depicts two separate models on the same set of data along with their respective squared errors. The actual data are represented by five black points, which are the exact same in each plot. The blue dashed line on the left plot represents a model learned using linear regression. The red dashed line on the right plot represents a model built from the mean of the sale price.

The error is the vertical distance from the black point to the dashed line. The squares in the plots represent the squared error of each point for each model. The sum of these squared errors (SSE) for each model can be found by adding up each of the squares. The image below shows this visually.

To calculate **R-squared** (written $R^2$), we divide the sum of squared errors from the model $SSE_{model}$ by the sum of squared errors from the mean $SSE_{mean}$. Finally, we subtract this number from 1.

![][0]

[0]: images/sse.png

$R^2$ represents the fractional decrease in error from the mean with respect to the model. To help clarify this, let's view the data, model predictions, and errors used in the visualizations above. For simplicity, the sale price is show in thousands.

| GrLivArea | SalePrice | $\hat{y}_{mean}$ | error mean | $\hat{y}_{model}$   | error model |
|-----------|-----------|------------|------------|---------|-------------|
| 1,150     | 185   | 198   | -13    | 178 | 7       |
| 1,500     | 180   | 198    | -18   | 189 | -9      |
| 1,800     | 195   | 198    | -3     | 198 | -3      |
| 2,100     | 210   | 198    | 12     | 207 | 3       |
| 2,450     | 220   | 198    | 22     | 218 | 2       |

Now, lets calculate the SSE for the model and the mean.

$$SSE_{model} = 7^2 + (-9)^2 + (-3)^2 + 3^2 + 2^2 = 152$$
$$SSE_{mean} = (-13)^2 + (-18)^2 + (-3)^2 + 12^2 + 22^2 = 1,130$$

$R^2$ can now be calculated.

$$R^2 = 1 - \frac{SSE_{model}}{SSE_{mean}} = 1 - \frac{152}{1,130} = .87$$

Our $R^2$ has been calculated as .87. Multiplying this number by, we can say that our model reduced the sum of squared error over guessing the mean by 87%.

Multiply $R^2$ by 100 gives us the percentage decrease in squared error.

$$R^2 = \frac{SSE_{red} - SSE_{blue}}{SSE_{red}}$$


### R-Squared is also percent of variance explained

The mean of the area of the red squares above is equivalent to statistical **variance**. $R^2$ is often reported as **the percentage of variance explained by the model**. The red squares are the natural amount of variance inherent in the data. A good model explains much of this variance.

Because the minimum amount of error is 0, the minimum value for $SSE_{model}$ is also 0. Thus, the maximum value for $R^2$ is 1 (100% of the variance is explained). Note that $R^2$ can be negative, which is counter intuitive because of the "squared" in its name, as the model you build can be worse than predicting just the mean. For instance, you can build a model that predicts 1 billion dollars for every single house, which would result in an enormous $SSE_{model}$ far greater that $SSE_{mean}$ forcing $R^2$ to be negative.

### Calculate $R^2$ for all coefficient combinations

Let's return to the full dataset and calculate $R^2$ for all the coefficient combinations from above. Below, we define a new function to return the $R^2$ for every combination as a DataFrame.

In [ ]:
def compute_r2(input_col, w0_range, w1_range):
    X = housing[input_col]
    y = housing['SalePrice']
    errors_r2 = {'w0': [], 'w1': [], 'r2': []}
    variance = ((y - y.mean()) ** 2).sum()
    for w0 in w0_range:
        for w1 in w1_range:
            y_pred = w0 + w1 * X
            sse = ((y - y_pred) ** 2).sum()
            r2 = (variance - sse) / variance
            errors_r2['w0'].append(w0)
            errors_r2['w1'].append(w1)
            errors_r2['r2'].append(r2)
    return pd.DataFrame(errors_r2)
        
df_errors_r2 = compute_r2('GrLivArea', w0_range, w1_range)
df_errors_r2.head()

### Sort from greatest to least

Our error metric, $R^2$, must be sorted in the opposite direction as RMSE. A higher value for $R^2$ indicates a better model.

In [ ]:
df_errors_r2.sort_values('r2', ascending=False).head()

### Best coefficients are the same

The best coefficient combination is the same as it was when using RMSE. This will always be the case, as both metrics are based on squared error.

### Visualize all combinations of $R^2$

We can pivot the DataFrame so we have a better view of all values of $R^2$ in a single table and then use pandas to highlight the background to show where the highest values occur.

In [ ]:
df_pivot = df_errors_r2.pivot(index='w0', columns='w1', values='r2').round(2)
df_pivot.style.background_gradient(cmap='coolwarm', axis=None)

## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">Repeat the same process for `GarageArea` and `LotFrontage`</span>

<span  style="color:green; font-size:16px">
    
1. Make a scatter plot of the input variable versus the sale price
2. Estimate the range for the y-intercept and slope and assign these to variables `w0_range` and `w1_range`
3. Use the `compute_r2` function
4. Visualize all the combinations of $R^2$ in by pivoting the resulting DataFrame and styling with a background gradient

</span>